In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL_BASE = 'https://www.argenprop.com'


def obtener_enlaces_desde_inicio():
    print("Obteniendo enlaces desde la página de inicio...")
    respuesta = requests.get(URL_BASE)
    sopa = BeautifulSoup(respuesta.content, 'html.parser')

    enlaces = []
    for h3 in sopa.find_all('h3', class_='btn btn-text btn-block'):
        etiqueta_a = h3.find('a')
        if etiqueta_a:
            href = etiqueta_a['href']
            if any(palabra in href for palabra in ["departamento-alquiler", "casa-alquiler", "inmuebles-alquiler"]):
                enlaces.append(href)

    print(f"Se obtuvieron {len(enlaces) - 3} enlaces desde la página de inicio.")
    return enlaces[3:]


def obtener_todos_enlaces_de_pagina(pagina_url):
    print(f"Obteniendo enlaces desde la página: {pagina_url}...")
    page_number = 1
    all_links = []
    while True:
        if page_number == 1:
            url = pagina_url
        else:
            url = pagina_url + "-pagina-" + str(page_number)

        respuesta = requests.get(url)

        if respuesta.status_code != 200:
            print(f"Error al obtener la página {page_number}. Código de estado: {respuesta.status_code}")
            break

        sopa = BeautifulSoup(respuesta.content, 'html.parser')
        enlaces_pagina = [enlace['href'] for enlace in sopa.find_all('a', class_='card') if enlace.has_attr('href')]

        if not enlaces_pagina:
            break

        all_links.extend(enlaces_pagina)
        page_number += 1

    print(f"Se obtuvieron {len(all_links)} enlaces desde la página: {pagina_url}.")
    return all_links


def extraer_datos_de_enlace(enlace):
    print(f"Extrayendo datos del enlace: {URL_BASE + enlace}...")
    respuesta = requests.get(URL_BASE + enlace)

    if respuesta.status_code != 200:
        print(f"Error al acceder al enlace: {enlace}. Código de estado: {respuesta.status_code}")
        return None

    soup = BeautifulSoup(respuesta.content, 'html.parser')

    try:
        location = soup.find('h3', class_='titlebar__address').text
        price = soup.find('p', class_='titlebar__price').text.strip()
        features_list_element = soup.find('ul', class_='property-main-features')
        features_list = features_list_element.find_all('li') if features_list_element else []
        features = {}
        for feature in features_list:
            key = feature.attrs['title']
            value = feature.find('div', class_='desktop').p.text.strip()
            features[key] = value
        expenses_element = soup.find('p', class_='titlebar__expenses')
        expenses = expenses_element.text.strip() if expenses_element else "No se encontraron expensas"

        data = {
            'URL': URL_BASE + enlace,
            'Ubicación': location,
            'Precio': price,
            **features,
            'Expensas': expenses
        }
        return data
    except Exception as e:
        print(f"Error al extraer datos del enlace: {enlace}. Error: {e}")
        return None


def main():
    enlaces_inicio = obtener_enlaces_desde_inicio()
    todos_enlaces = []

    for enlace in enlaces_inicio:
        todos_enlaces.extend(obtener_todos_enlaces_de_pagina(enlace))

    datos_lista = []
    for enlace in todos_enlaces:
        data = extraer_datos_de_enlace(enlace)
        if data:
            datos_lista.append(data)

    df = pd.DataFrame(datos_lista)
    df.to_csv('propiedades_argenprop.csv', index=False)
    print("¡Datos guardados exitosamente en 'propiedades_argenprop.csv'!")

    # Para visualizar el dataframe
    print(df.head())


if __name__ == "__main__":
    main()